In [1]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from keras.optimizers import Adam
from keras.preprocessing import image
from keras import backend as K
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [132]:
K.clear_session()

In [2]:
vgg_base = VGGFace(include_top=False, input_shape=(224, 224, 3), pooling='max')

In [10]:
# vgg_base.summary()

<h1> First change applies to architecture </h1>

## We use K sigmoid output units in the last layer, and also binary crossentropy for loss.  ##

In [4]:
# Add custom layers
last_layer = vgg_base.get_layer('global_max_pooling2d_1').output
#X = Dropout(0.2)(last_layer)
X = Dense(128, activation='relu', name='fc6')(X)
X = Dropout(0.2)(X)
X = Dense(64, activation='relu', name='fc7')(X)
X = Dropout(0.2)(X)
output = Dense(6, activation='sigmoid')(X)

In [5]:
# this is the model we will train
model = Model(inputs=vgg_base.input, outputs=output)

In [6]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in vgg_base.layers:
    layer.trainable = False

In [7]:
# Make sure weights are not trainable
model.trainable_weights

[<tf.Variable 'fc6/kernel:0' shape=(512, 128) dtype=float32_ref>,
 <tf.Variable 'fc6/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'fc7/kernel:0' shape=(128, 64) dtype=float32_ref>,
 <tf.Variable 'fc7/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(64, 6) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(6,) dtype=float32_ref>]

In [8]:
model.compile(optimizer=Adam(clipnorm=1.0), 
               loss='binary_crossentropy', 
               metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [11]:
BASE_DIR = 'C:\\Users\\Lenovo 500\\Desktop\\Projects\\LoveClassifier\\all_females'

In [12]:
# IMPORTANT !
# LISTDIR returns the files as indexed by the filessystem
# not necessarily in alphanumeric order, so need to sort them!
N_IMAGES = len(os.listdir(BASE_DIR))
IMAGE_SIZE = (224, 224)

X_train = np.zeros((N_IMAGES, IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
y_train = np.zeros(N_IMAGES)

for idx, _im in enumerate(sorted(os.listdir(BASE_DIR))):
    # Change the image path with yours.
    _img = image.load_img(os.path.join(BASE_DIR, _im), target_size=IMAGE_SIZE)
    _x = image.img_to_array(_img)
    _x = np.expand_dims(_x, axis=0)
    X_train[idx, :, :, :] = utils.preprocess_input(_x, version=1) / .255 

In [126]:
# Make sure images are ok
assert ~np.any(np.isnan(X_train))

In [13]:
# Load ratings
ratings = np.genfromtxt('./Projekt_SGE_Assessment_ErikK.txt')

<h1> Here comes the big change. </h1> 
### Since we need to do ordered regression, we need some way of forcing the network to learn that 0 < 1 < 2 < 3 < 4 < 5. According to Cheng (2007), we can represent the ordered classes like this:
### [0] -> [0, 0, 0, 0, 0]
### [1] -> [1, 0, 0, 0, 0]
### [2] -> [1, 1, 0, 0, 0]  
### [3] -> [1, 1, 1, 0, 0]
### [4] -> [1, 1, 1, 1, 0]
### [5] -> [1, 1, 1, 1, 1]
### And use K sigmoid output units at the last layer. Let's check it out

In [24]:
# Convert ratings to the desired format
# 1. First round them
ratings_rounded = np.round(ratings, 1).astype(np.int8)

# 2. Then encode as suggested by Cheng (2007)
ratings_prepared = np.zeros((len(ratings_rounded), len(np.unique(ratings_rounded))))
for i, r in enumerate(ratings_rounded):
    for j in range(r):
        ratings_prepared[i, j] = 1
        
# 3. Finally, make sure assignment is correct
assert np.all(np.sum(ratings_prepared, axis=1).astype(np.int8) == ratings_rounded)

In [ ]:
history = model.fit(X_train, 
                    ratings_prepared, 
                    batch_size=64, 
                    epochs=4, 
                    validation_split=0.1)

Train on 511 samples, validate on 57 samples
Epoch 1/4
511/511 [==============================] - 572s - loss: 4.2125 - acc: 0.7374 - val_loss: 4.5244 - val_acc: 0.7193
Epoch 2/4
511/511 [==============================] - 607s - loss: 4.0658 - acc: 0.7466 - val_loss: 3.3379 - val_acc: 0.7924
Epoch 3/4


<h1> Prediction </h1>
### According to Cheng (2007, p. 3): 
"In the test phase, to make a prediction, our method
scans output nodes in the order O1, O2, ..., OK. It
stops when the output of a node is smaller than the
predeﬁned threshold T (e.g., 0.5) or no nodes left. The
index k of the last node Ok whose output is bigger than
T is the predicted category of the data point."